# 4. 거시경제 변수 생성

In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
data = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Seoul_location/종로구 실거래(13~18).xlsx")
GDP = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Factor/국내총생산 및 경제성장률 (GDP)_2013~2018.xlsx")
prices = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Factor/월별 소비자 물가지수_2013~2018.xls")
rate = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Factor/일별 평균 환율 조회_2013~2018.xlsx")
kospi = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Factor/주가 종합 지수_2013~2018.xls")

## 4-1 국내총생산, 경제성장률 변수 생성

In [3]:
# data 파일에 분기 변수 생성
data['분기'] = ''

for i in range(len(data)):
    if data.at[i,'월']<4:
        data.at[i,'분기'] = str(data.at[i,'년']) + '1/4'
    elif data.at[i,'월']>3 and data.at[i,'월']<7:
        data.at[i,'분기'] = str(data.at[i,'년']) + '2/4'
    elif data.at[i,'월']>6 and data.at[i,'월']<10:
        data.at[i,'분기'] = str(data.at[i,'년']) + '3/4'
    else:
        data.at[i,'분기'] = str(data.at[i,'년']) + '4/4'

In [4]:
# 국내총생산, 경제생산률 매칭
a = data['분기'].values
b = GDP['분기'].values
data['E_GDP'] = ''
data['E_grow'] = ''

for i in range(len(data)):
    for j in range(len(GDP)):
        if a[i] == b[j]:
            data.at[i,'E_GDP'] = GDP.at[j,'국내총생산']
            data.at[i,'E_grow'] = GDP.at[j,'경제성장률']
            continue

data.drop(['분기'], axis='columns', inplace=True)

## 4-2 소비자물가, 집세 변수 생성

In [5]:
# data 파일에 시기 변수 생성
data['시기'] = 0

for i in range(len(data)):
    data.at[i,'시기'] = str(data.at[i,'년']) + str(data.at[i,'월'])

In [6]:
# 소비자물가, 집세 매칭
a = list(data['시기'])
b = list(prices['시기'])
c = list(kospi['시기'])
data['E_CPI'] = ''
data['E_rent'] = ''
data['E_kospi'] = ''

for i in range(len(data)):
    if a[i] in b:
        x = b.index(a[i])
        data.at[i,'E_CPI'] = prices.at[x,'소비자물가']
        data.at[i,'E_rent'] = prices.at[x,'집세']
    
    if a[i] in c:
        y = c.index(a[i])
        data.at[i,'E_kospi'] = kospi.at[y,'코스피']
            

data.drop(['시기'], axis='columns', inplace=True)

## 4-3 환율 변수 생성

In [7]:
# data 파일에 일자 변수 생성
data['일자'] = ''

for i in range(len(data)):
    data.at[i,'일자'] = str(data.at[i,'년']) + '.' + str(data.at[i,'월']) + '.' + str(data.at[i,'일'])

In [8]:
#data 날짜 형식 변경
data["일자"] = pd.to_datetime(data["일자"])

In [9]:
# 가까운 날짜 찾기 함수
def nearest(day, rate):
    M = {}
    
    for i in range(len(rate)):
        if day.year == rate.at[i,'일자'].year and day.month == rate.at[i,'일자'].month:
                M[abs(day.day - rate.at[i,'일자'].day)] = i
    x = min(M.keys())
    
    return(rate['한국은행기준율'][M[x]])

In [10]:
# 소비자물가, 집세 매칭(같은 날짜 있는 것)

a = list(data['일자'])
b = list(rate['일자'])
data['E_rate'] = ''

for i in range(len(data)):
    if a[i] in b:
        x = b.index(a[i])
        data.at[i,'E_rate'] = rate.at[x,'한국은행기준율']
        continue

In [11]:
# 소비자물가, 집세 매칭(같은 날짜 없는 것)

a = data['E_rate'].values

for i in range(len(data)):
    if a[i] == '':
        data.at[i,'E_rate'] = nearest(data.at[i,'일자'], rate)
                
data.drop(['일자'], axis='columns', inplace=True)

In [12]:
data.to_excel('종로구 실거래(13~18).xlsx', index = False)